## Semantic Segmentation with Masterful 

In this guide we will walk you through the steps on how to use the Masterful AutoML platform to train a model on the task of semantic segmentation. 

This guide is inspired by the Tensorflow [Image Segmentation](https://www.tensorflow.org/tutorials/images/segmentation) guide. We use the same dataset ([Oxford IIIT Pets](https://www.robots.ox.ac.uk/~vgg/data/pets/)) and model (a modified [U-Net](https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/)) as that guide, so that you can see the side-by-side comparison between training with and without Masterful.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/masterfulai/masterful-docs/blob/main/notebooks/guide_semantic_segmentation.ipynb)
[![download button](https://www.tensorflow.org/images/download_logo_32px.png)](https://masterful-public.s3.us-west-1.amazonaws.com/933013963/latest/guide_semantic_segmentation.ipynb) Download this Notebook

### Import the Necessary Libraries

In this Guide, we will be using Tensorflow as the training infrastructure, and Tensorflow Datasets to provide the training data. We use Keras on top of Tensorflow to help us build the model, and since we are using the same model as the [Image Segmentation](https://www.tensorflow.org/tutorials/images/segmentation) guide, we have a few imports to pull in the model definition from there. 

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

# We use Keras to build the individual model layers
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

### Register Masterful

Registration is necessary to ensure you have a valid authorization key and are licensed to use the Masterful AutoML platform. Make sure to fill in the `<INSERT HERE>` sections with the account ID and authorization key provided to you by Masterful AI.

In [2]:
MASTERFUL_ACCOUNT_ID="<INSERT HERE>"
MASTERFUL_AUTHORIZATION_KEY="<INSERT HERE>"

import masterful
masterful = masterful.register(account_id=MASTERFUL_ACCOUNT_ID, authorization_key=MASTERFUL_AUTHORIZATION_KEY)

MASTERFUL: Your account has been successfully registered. Masterful v0.3.6 is loaded.


### Prepare the Dataset

The dataset is available from [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/oxford_iiit_pet). The segmentation masks are included in version 3+.

In [3]:
INPUT_SHAPE = (128, 128, 3)
dataset_splits, info = tfds.load('oxford_iiit_pet:3.*.*', with_info=True, split=['train', 'test'])

Tensorflow Datasets returns a dictionary of features for each example, but Masterful requires an explicit (image, label) tuple for input examples, similar to the input you would provide to Keras [model.fit()](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit), for example. Therefore, the first task is to extract the image and label (in this case the segmentation mask) from the feature dictionary.

After the image and label has been extracted, we need to standardize the data for the model we are using. In this case, the U-Net model we have chosen expects each input image to be in a square format. In addition, the image color space should be RGB floats in the range [0,1].

Finally, we need to create the labels (segmentation masks) we will use for supervised training. The Oxford Pets dataset consists of images of 37 pet breeds, with 200 images per breed (~100 each in the training and test splits). Each image includes the corresponding labels, and pixel-wise masks. The masks are class-labels for each pixel. Each pixel is given one of three categories:

- Class 1: Pixel belonging to the pet.
- Class 2: Pixel bordering the pet.
- Class 3: None of the above/a surrounding pixel. 

For convenience, we will convert the class labels from [1,2,3] to [0,1,2] similar to the [Image Segmentation](https://www.tensorflow.org/tutorials/images/segmentation) guide, and we will then convert them to one-hot class labels, because Masterful performs better with dense rather than sparse labels. 

In [4]:
def extract_image_and_label(features):
  """ 
  Extracts the image and segmentation mask from the feature dictionary,
  and applies minimal normalization (resizing and label standardization).
  """
  image = tf.image.resize(features['image'], (INPUT_SHAPE[0], INPUT_SHAPE[1]))
  mask = tf.image.resize(features['segmentation_mask'], (INPUT_SHAPE[0], INPUT_SHAPE[1]))

  # Convert the image into the [0,1] RGB color space.
  image = tf.cast(image, tf.float32) / 255.0

  # For convenience, convert the segmentation mask into 
  # [0,1,2] class labels.
  mask -= 1

  # Convert to one-hot labels in the mask.
  mask = tf.one_hot(tf.cast(tf.squeeze(mask, axis=-1), tf.int32), depth=3)      
  return image, mask

Make sure to apply the `extract_image_and_label` function to both the training and test datasets, so that model training and evaluation both see the same input.

In [6]:
labeled_dataset = dataset_splits[0]
test_dataset = dataset_splits[1]

labeled_dataset = labeled_dataset.map(extract_image_and_label, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(extract_image_and_label, num_parallel_calls=tf.data.AUTOTUNE)

# The Oxford Pets dataset on TF Datasets contains a few corrupted images,
# as well as several images that do not decode correctly. We apply
# the ignore_errors() operator here to eliminate the annoying warnings
# that will show up in the console due to these errors.
labeled_dataset = labeled_dataset.apply(tf.data.experimental.ignore_errors())
test_dataset = test_dataset.apply(tf.data.experimental.ignore_errors())

### Define the Model (U-Net)

The model being used here is a modified [U-Net](https://arxiv.org/abs/1505.04597). A U-Net consists of an encoder (downsampler) and decoder (upsampler). In this guide, we are using a very simple encoder/decoder architecture, to demonstrate the principles of using Masterful. This model should not be used in a production environment.

In [8]:
def simple_unet(input_shape):
  """
  Creates a simple UNet encoder-decoder architecture, with a
  3 channel output (logits based) for semantic segmentation.
  Assumes an input/output size of (128,128).
  """

  def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

  def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

  def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

  inputs = Input(input_shape)

  s1, p1 = encoder_block(inputs, 32)
  s2, p2 = encoder_block(p1, 64)
  s3, p3 = encoder_block(p2, 128)
  s4, p4 = encoder_block(p3, 128)

  b1 = conv_block(p4, 128)

  d1 = decoder_block(b1, s4, 128)
  d2 = decoder_block(d1, s3, 128)
  d3 = decoder_block(d2, s2, 64)
  d4 = decoder_block(d3, s1, 32)

  outputs = Conv2D(3, 1, padding="same")(d4)
  model = Model(inputs, outputs, name="U-Net")
  return model
model = simple_unet(INPUT_SHAPE)

### Train the Model.

The first step when using Masterful is to specify some design choices about your model by using `masterful.spec.create_model_and_data_specs`. Masterful tries to infer as much as possible about your model and dataset, but for things which cannot be inferred, Masterful has created `masterful.ModelSpec` and `masterful.DataSpec` data structures to inform the platform about these choices. 

In the example below, we are letting Masterful know that we are performing a semantic segmentation task (`masterful.spec.Task.SEMANTIC_SEGMENTATION`) with 3 class labels (`num_classes=3`), and that our input image features are in the range [0,1] (`image_range=masterful.spec.ImageRange.ZERO_ONE`). Furthermore, we are providing dense labels (`sparse=False`) and that our model is outputting logits rather than a softmax classification (`from_logits=True`). 

In [15]:
model_spec, data_spec = masterful.spec.create_model_and_data_specs(
    model=model,
    dataset=labeled_dataset,
    task=masterful.spec.Task.SEMANTIC_SEGMENTATION,
    image_range=masterful.spec.ImageRange.ZERO_ONE,
    num_classes=3,
    sparse=False,
    from_logits=True, 
)

Now, you are ready to train your model using the Masterful AutoML platform. In the next cell, you will see the call to `masterful.autofit`, which is the entry point to the meta-learning engine of the Masterful AutoML platform. Notice there is no need to batch your data (Masterful will find the optimal batch size for you). No need to shuffle your data (Masterful handles this for you). You don't even need to pass in a validation dataset (Masterful finds one for you). You hand Masterful a model and a dataset, and Masterful will figure the rest out for you.

In [16]:
fit_policy, fit_report = masterful.autofit(
  model, 
  model_spec, 
  labeled_dataset, 
  data_spec,
)

MASTERFUL: Auto-fitting model to datasets.
...
MASTERFUL: Training complete in 13.6272061546643575 minutes.


The model you passed into `masterful.autofit` is now trained and updated in place, so you are able to evaluate it just like any other trained Keras model.

In [ ]:
model.evaluate(test_dataset.batch(fit_policy.batch_size))

58/58 [==============================] - 3s 49ms/step - loss: 0.2624 - categorical_accuracy: 0.9003
